# Run PGW-software on JupyterLab

This notebook can be used by users with a CSCS account to prepare PGW-simulations. It has been developed to be run on https://jupyter.cscs.ch/ on one node.

## Prerequisites

1. The software is written in python and has various dependencies that are listed in [pgw_conda_env.yml](./pgw_conda_env.yml). The dependencies must be available in a kernel in jupyterlab. The commands to install such a kernel on can be found in [/Documentations/Howto_install_kernel_CSCS.md](./Documentations/Howto_install_kernel_CSCS.md) (installing on /project or /store prevents issues with the memory limit in your home). 
1. Monthly mean changes for the desried climatic variables or climate change deltas from GCMs or RCMs are needed. You find instructions on how to do that in [/Documentations/README_CMOR.md](./Documentations/README_CMOR.md).

## Step 1: Interpolate input data to output time frequency

First we define paths to input data (netcdf format) as a list. We also provide the name of the variables within the netcdf file as a list. Then using the variable output_time_steps, we give the amount of timesteps we need as output in one year (366 * 4 for 6-hourly). The scripts only work if we produce output for the entire year even if only part of it is needed. Also we have to provide the loaction to save the output of the function. 

In [3]:
samepath='/scratch/snx3000/robro/pgwtemp/deltas/GCMdata/'

file_path_int = [
samepath+'diff_hur.nc',
samepath+'diff_hurs.nc',
samepath+'diff_ta.nc',
samepath+'diff_tas.nc',
samepath+'diff_ua.nc',
samepath+'diff_va.nc',
]

variablename = ['hur', 'hurs', 'ta', 'tas', 'ua', 'va']

output_time_steps = 366 * 4
inputfreq = 'month'
outputpath_int = '/scratch/snx3000/robro/pgwtemp/interpolated/'

Next we import all necessary modules and initialize a dask distributed client for paralell execution. Sometimes this gives an "Exception", but can be ignored if the client runs...

In [ ]:
import dask
from dask.distributed import Client
from distributed.diagnostics.plugin import UploadFile
import os
from interpolate import interpannualcycle_dask #custom function (needs to be in same folder as notebook)

threads_per_task = 4
client = Client(n_workers=6, threads_per_worker=threads_per_task) #one worker per variable
print(client)# see if clinet runs

Now we define the tasks we want to run (the function interpannualcycle for every variable):

In [ ]:
tasks=[]
wordir=os.getcwd()

for path_in, variable_in in zip(file_path_int, variablename):
    temp = dask.delayed(interpannualcycle_dask)(path_in, variable_in,\
            output_time_steps, inputfreq, outputpath_int, threads_per_task)
    tasks.append(temp)

tasks #should show all tasks

Now we send the tasks to the cluster for computation (we also need to make all the workers aware of the custom python function). We are using the dask version to avoid memory issues. This will take some time (2-3h), make sure you have assigned enough time in Jupyterlab and do something else in between (log out possible)! Whenever there are no memory issues the original numpy-based code will be faster due to IO.

In [ ]:
client.register_worker_plugin(UploadFile(wordir+"/interpolate.py"))
dask.compute(*tasks, scheduler='processes')

## Step 2: Regrid to regional model grid horizontally

After the inperpolation to the new timesteps is finished, we can continue on the single netcdf-files saved and regrid them to the grid of the regional model. This requires new settings. First, a path to a netcdf file using the target grid must be specified (has to include the lat and lon coordinates of the target). Second, a folder where the output can be stored should be given.

In [4]:
outputgridfile = '/store/c2sm/ch4/robro/surrogate_input/lffd2005112000c.nc'
outputfolder_regrid = '/scratch/snx3000/robro/pgwtemp/regridded/'

#take previously defined values:
infolder = outputpath_int
inputtimesteps = output_time_steps

Next we will prepare the tasks to run on the cluster, just as as we have done in the first step. Many commands are just repeated for savety reasons.

In [ ]:
import dask
from dask.distributed import Client
from distributed.diagnostics.plugin import UploadFile
import os
from regrid_horizontal import regridhorizontal #custom function (needs to be in same folder as notebook)

client = Client(n_workers=6, threads_per_worker=4) #one worker per variable
print(client)# see if clinet runs

tasks=[]
wordir=os.getcwd()

for variable in variablename:
    temp = dask.delayed(regridhorizontal)(infolder, variable, inputtimesteps, outputgridfile, outputfolder_regrid)
    tasks.append(temp)

tasks #should show all tasks

Now send the tasks to the cluster. This should run faster than the previous step but can also take some time.

In [ ]:
client.register_worker_plugin(UploadFile(wordir+"/regrid_horizontal.py"))
dask.compute(*tasks, scheduler='processes')

## Step 3: Change vertical coordinate for 3D fields (CCLM_only)

Lastly, we need to change the vertical coordinate to that of the regional model. This is highly dependent on the regional model used and no generic solution exists... The code here is specific to COSMO-CLM.<br>
We first again need to give some settings.

In [5]:
terrainpath = outputgridfile
datapath = outputfolder_regrid

variablename = ['hur','ta','ua','va'] #only 3D data
out_vars = ['RELHUM', 'T', 'U', 'V'] #rename variable to the correct ones for cosmo
outputpath_vertical = '/scratch/snx3000/robro/pgwtemp/final2/'

vcflat = 11357 #height where modellevels become flat (see cosmo namelist)

steps_per_job = inputtimesteps + 100 #this option is not used in notebook just leave as it is
starttime = 0 #set to 0; not used in notebook

<div class="alert alert-block alert-danger">
<b>Important:</b> The vertical level information is read from the file <b>heights.txt</b> which needs to be checked and updated with own level specifications. It can be copy-pasted from an YUSPECIF file from a COSMO simulation using the target vertical levels.
</div>

In [ ]:
import numpy as np
print('The following height half-levels will be used, should match namelist')
print(np.genfromtxt('heights.txt',skip_header=1)[:,1])

As before we define the tasks to run. Note that we need less workers than before as we have less variables. Thus we may have to close dask.distributed clients used before.

In [ ]:
import dask
from dask.distributed import Client
from distributed.diagnostics.plugin import UploadFile
import os
from cclm_vertical import vertinterpol #custom function (needs to be in same folder as notebook)

try:
    client.close()
except:
    pass

client = Client(n_workers=4, threads_per_worker=6) #one worker per variable
print(client)# see if clinet runs

tasks=[]
wordir=os.getcwd()

for variable,out_var in zip(variablename, out_vars):
    temp = dask.delayed(vertinterpol)(terrainpath, datapath, variable, out_var, \
                                      outputpath_vertical, vcflat, steps_per_job, starttime)
    tasks.append(temp)

tasks #should show all tasks

AAAAND submit :), This will again take a couple of hours (around 3h on one node), it is save to logout...

In [ ]:
client.register_worker_plugin(UploadFile(wordir+"/cclm_vertical.py"))
dask.compute(*tasks, scheduler='processes')

## Start of Postprocessing - Step 1: Rename 2D variables

Now we have computed the climate change deltas :). Now we can add them to the initial condition and lateral boundary condition files from the control simulation that were created by int2lm. As a first step we quickly rename the 2D variables from the CMOR naming convention to what is used by COSMO. Most likely you will have to re-run all cells above with definitions of paths to avoid errors.

Define the names to be renamed as lists:

In [6]:
#take previously defined variables
output_path_new_2D = outputpath_vertical
difference_2d_files_path = outputfolder_regrid
output_time_steps = output_time_steps

variablename_cmor = [ 'hurs', 'tas' ]
variablename_cclm = ['RELHUM_S', 'T_S'] #tas is modifiyed to T_S instead of T_2M to adapt the sea surface temps

Loop over the list and execute the renaming (should be fast):

In [ ]:
for var_cmor, var_cclm in zip(variablename_cmor, variablename_cclm):
    %run Postprocess_CCLM/rename_variables.py $var_cmor $var_cclm $difference_2d_files_path $output_path_new_2D $output_time_steps

<div class="alert alert-block alert-warning">
<b>Warning:</b> At this state it makes sense to move all climate changes deltas that were computed (3D and 2D) to a permanent file system so you can access them later.
</div>

## Postprocessing Step 2 - adapt initial condition file

We are now ready to adapt the initial condition file of the RCM and subsequently the boundary condition files. These need to be present on the file system. 

<div class="alert alert-block alert-info">
<b>Information:</b> Before running this step you need to run int2lm for the historical/evaluation period and/or have the laf*.nc and lbfd*.nc files ready on the cluster.
</div>

Information as input for the file needs to be provided next (no computation is triggered). 
1. path_to_deltas: the path where the climate change deltas created by previous scripts are saved
1. output_path: the folder where the output should be saved (can be the same as above, the filename is changed). 
1. constant_variables_file: We again need information on the terrainheight from a constant field (for computation of reference pressure).
1. lbfds_files_path: Where are the inintial/boundary conditions files (Int2lm output)?
1. changeyears: How many years should be added to the model calendar (determines greenhouse-gas concentration in COSMO).
1. laftimestring: A string giving the future timestep of the laf in the format 'seconds since yyyy-mm-dd hh:mm:ss'
1. starttimestep: What timestep within the yearly cycle has to be used for the initial conditions? (0 = midnight jaunuary first; otherwise dayofyear * boundary update frequency)
1. recompute_pressure: Should the pressure be recomputed to maintain the hydrostatic balance after the temperature is changed? True or False. If set to false a climate change delta for PP (deviation from reference pressure in cosmo) must be provided.

In [11]:
import glob
path_to_deltas = output_path_new_2D

constant_variables_file = outputgridfile

lbfds_files_path='/scratch/snx3000/robro/PGW_atl_04/output/ifs2lm'
#this might have to be changed if lbfd files are organized according to years (the date will be changed by this script)
output_path=lbfds_files_path

# (no need to change this block) get laf file path and year by reading from above directory
laf_file=glob.glob(f'{lbfds_files_path}/laf*')
laf_file.sort()
laf_file = laf_file[0]
year_laf=laf_file.split('laf')[1][:4]

#how many years to add laf files? #for nomal calendars should be able to devide by 4 due to leap years (needed for adjustment of CO2 concentrations)
changeyears=88
year_laf = int(year_laf) + changeyears

#put the correct start date
laftimestring = f'seconds since {year_laf}-11-01 00:00:00'
starttimestep=305 * 4 #0 for 1st of january then count up depending on boundary update frequency

recompute_pressure=True #True if no pressure change are available as as climate change deltas; false if a PP-Diff file exists (if input comes from CCLM)

print('using this file: ', laf_file)

using this file:  /scratch/snx3000/robro/PGW_atl_04/output/ifs2lm/laf2004110100.nc


Run the respective script directly, as it works on one file only it should not take long. 

In [24]:
%run Postprocess_CCLM/laf_adapt.py $laf_file $year_laf $output_path $path_to_deltas $constant_variables_file "$laftimestring" $starttimestep $recompute_pressure

saved /scratch/snx3000/robro/PGW_atl_04/output/ifs2lm/laf2092110100.nc


<div class="alert alert-block alert-info">
<b>Tipp:</b> Do a quick sanity check to see if the original laf and the adapted laf file can are different as expected. e.g. using CDO in a terminal: cdo diffn 'name of original laf'.nc 'name of new laf'.nc
</div>

## Postprocessing Step 3 - adapt boundary condition files

Now for the heavy lifting in the postprocessing: Add the derived climatic changes to all boundary condition files. Note that the same changes are need to be added for each simulation year. Most of the information needed for the computation is that same as for the initial condition file.

In [43]:
#make sure the right paths are taken from adaption of laf
lbfds_files_path = lbfds_files_path
output_path = output_path

path_to_deltas = path_to_deltas
input_timesteps_deltas = inputtimesteps

constant_variables_file = constant_variables_file

starttimestep = starttimestep
changeyears = changeyears

recompute_pressure = recompute_pressure

#If you have lbfd files for more than one year you might want to loop over different years, 
#otherwise leave st_year and end_year at same random number
st_year=2004
end_year=2004


Now we again initialize a dask cluster. Use one worker per year you want to run the PGW for (or just do it manually multiple times).

In [44]:
import dask
from dask.distributed import Client
from distributed.diagnostics.plugin import UploadFile
import os
from Postprocess_CCLM.lbfd_adapt import workflow_lbfd #custom function (needs to be in same folder as notebook)

wordir=os.getcwd()

try:
    client.close()
except:
    pass

client = Client(n_workers=1, threads_per_worker=24) #one worker per year (maximum 24 on piz daint)
print(client)# see if clinet runs

tasks=[]

for yyyy in range(st_year, end_year + 1):
    newyear = yyyy + changeyears
    
    if end_year - st_year > 0: # if one wants to sumbit multiple jobs for different years, files should be grouped in folders for the year
        output_path = f'{output_path}/{newyear}/'
        lbfds_files_path = f'{lbfds_files_path}/{yyyy}/'
    
    os.makedirs(output_path, exist_ok=1)
    
    temp = dask.delayed(workflow_lbfd)(yyyy, lbfds_files_path, output_path, path_to_deltas, constant_variables_file, starttimestep,\
                                       input_timesteps_deltas, changeyears, recompute_pressure)
    tasks.append(temp)

tasks #should show all tasks

<Client: 'tcp://127.0.0.1:42017' processes=1 threads=24, memory=63.96 GB>


[Delayed('workflow_lbfd-2185e299-cb3e-46bf-9e51-fc31a6208baa')]

In [ ]:
client.register_worker_plugin(UploadFile(wordir+"/Postprocess_CCLM/lbfd_adapt.py"))
dask.compute(*tasks, scheduler='processes')

<div class="alert alert-block alert-success">
<b>Success:</b> After the above command finished you should be ready to run CCLM. Again it is recommended to do a quick sanity check of the output files in the command line.
</div>